In [8]:
import socket
import threading

class ClientHandler(threading.Thread):

    def __init__(self, conn, addr, chat_server):
        threading.Thread.__init__(self)
        self.conn = conn
        self.addr = addr
        self.server = chat_server
        self.name = f"Cliente-{addr[0]}:{addr[1]}"
        self.nombre_usuario = None

    def run(self):

        try:
            data = self.conn.recv(1024)
            if not data:
                print(f"[CONEXIÓN CERRADA] {self.name} se desconectó antes de enviar su nombre.")
                self.conn.close()
                return

            self.nombre_usuario = data.decode('utf-8').strip()

            if not self.nombre_usuario:
                print(f"[CONEXIÓN RECHAZADA] {self.name} envió un nombre vacío.")
                self.conn.sendall("Error: El nombre de usuario no puede estar vacío.\n".encode('utf-8'))
                self.conn.close()
                return

            print(f"[NUEVA CONEXIÓN] {self.name} se identificó como: {self.nombre_usuario}")

            mensaje_bienvenida = f"--- {self.nombre_usuario} se ha unido al chat ---\n"
            self.server.broadcast(mensaje_bienvenida, self.conn)

            while True:
                data = self.conn.recv(1024)

                if not data:
                    break

                mensaje = data.decode('utf-8').strip()

                mensaje_broadcast = f"[{self.nombre_usuario}]: {mensaje}\n"

                print(mensaje_broadcast, end='')

                self.server.broadcast(mensaje_broadcast, self.conn)

        except ConnectionResetError:
            print(f"[ERROR] {self.nombre_usuario} ({self.name}) se desconectó abruptamente.")
        except Exception as e:
            print(f"[ERROR] Error con {self.nombre_usuario} ({self.name}): {e}")
        finally:
            self.conn.close()

            self.server.remove_client(self.conn)

            if self.nombre_usuario:
                print(f"[CONEXIÓN CERRADA] {self.nombre_usuario} ({self.name}) se ha desconectado.")

                mensaje_salida = f"--- {self.nombre_usuario} ha salido del chat ---\n"
                self.server.broadcast(mensaje_salida, self.conn)
            else:
                print(f"[CONEXIÓN CERRADA] {self.name} se desconectó antes de identificarse.")

class ChatServer:

    def __init__(self, host, port):
        self.clients = []
        self.clients_lock = threading.Lock()
        self.sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.server_address = (host, port)
        self.sock.bind(self.server_address)

    def add_client(self, conn):
        with self.clients_lock:
            self.clients.append(conn)

    def remove_client(self, conn):
        with self.clients_lock:
            if conn in self.clients:
                self.clients.remove(conn)

    def broadcast(self, message, sender_conn):
        with self.clients_lock:
            for client_conn in self.clients:
                if client_conn != sender_conn:
                    try:
                        client_conn.sendall(message.encode('utf-8'))
                    except Exception as e:
                        print(f"[ERROR BROADCAST] No se pudo enviar a {client_conn.getpeername()}: {e}")

    def run(self):
        self.sock.listen(5)
        print('Iniciando servidor en {} puerto {}'.format(*self.server_address))

        try:
            while True:
                conn, addr = self.sock.accept()

                self.add_client(conn)

                handler_thread = ClientHandler(conn, addr, self)

                handler_thread.start()

        except KeyboardInterrupt:
            print("\n[APAGANDO] Cerrando el servidor...")
        finally:
            with self.clients_lock:
                for client_conn in self.clients:
                    client_conn.close()
            self.sock.close()
            print("[SERVIDOR APAGADO]")

if __name__ == "__main__":
    HOST = 'localhost'
    PORT = 8070

    server = ChatServer(HOST, PORT)
    server.run()

Iniciando servidor en localhost puerto 8070
[NUEVA CONEXIÓN] Cliente-127.0.0.1:54714 se identificó como: Daner
[Daner]: Holaaa
[Daner]: Como estan
[CONEXIÓN CERRADA] Daner (Cliente-127.0.0.1:54714) se ha desconectado.

[APAGANDO] Cerrando el servidor...
[SERVIDOR APAGADO]
